## Example workbook - highly composite derivations

This notebook is an extension of hands-on session 2.4 including the ion species in the temperature model. For a more pedagogical explanation, see that workbook.

In [1]:
import numpy as np

import holoviews as hv
import matplotlib.pyplot as plt
import panel as pn

import anisotropy as aniso
import RMK_support.simple_containers as sc
import RMK_support.common_models as cm 
import RMK_support.IO_support as io
from RMK_support import RKWrapper, Grid, treeDerivation, Node, erf
import RMK_support.dashboard_support as ds

## Basic setup

In [2]:
#Wrapper initialization
rk = RKWrapper()

#I/O setuo
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/day_2_4/"
rk.setHDF5Path(hdf5Filepath) 

#Grid initialization
xGridWidths = np.ones(2)
rk.grid = Grid(xGridWidths, interpretXGridAsWidths=True)

## Normalisation

In [3]:
elCharge = 1.60218e-19
elMass = 9.10938e-31 # electron mass unit
amu = 1.6605390666e-27  # atomic mass unit
heavySpeciesMass = 2.014  # in amus
ionMass = heavySpeciesMass*amu  # deuterium mass

rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

## Handling particle species data

In [4]:
rk.addSpecies("e", speciesID=0, atomicA=elMass/amu, charge=-1.0, associatedVars=["ne", "Ge", "WePar", "WePerp"])
rk.addSpecies("D+", speciesID=-1, atomicA=heavySpeciesMass, charge=1.0, associatedVars=["ni", "Gi", "WiPar", "WiPerp"])

electronSpecies = rk.getSpecies("e")
ionSpecies = rk.getSpecies("D+")

## Base variables 

In [5]:
TePar = 2*np.ones(2)
TePerp = 5*np.ones(2)
TiPar = 8*np.ones(2)
TiPerp = 10*np.ones(2)

ne = np.ones(2)
ni = np.ones(2)

WePar = ne*TePar/2
WePerp = ne*TePerp
WiPar = ni*TiPar/2
WiPerp = ni*TiPerp

In [6]:
# Implicit variables
rk.addVarAndDual("ne", ne, units='$10^{19} m^{-3}$', isCommunicated=True)
rk.addVarAndDual("ni", ni, units='$10^{19} m^{-3}$', isCommunicated=True)
rk.addVarAndDual("Ge", primaryOnDualGrid=True, isCommunicated=True)
rk.addVarAndDual("Gi", primaryOnDualGrid=True, isCommunicated=True)
rk.addVarAndDual("WePar", WePar, units='$10^{20} eV m^{-3}$', isCommunicated=True)
rk.addVarAndDual("WePerp", WePerp, units='$10^{20} eV m^{-3}$', isCommunicated=True)
rk.addVarAndDual("WiPar", WiPar, units='$10^{20} eV m^{-3}$', isCommunicated=True)
rk.addVarAndDual("WiPerp", WiPerp, units='$10^{20} eV m^{-3}$', isCommunicated=True)

# Temperatures
rk.addVarAndDual("TePar", TePar, isStationary=True, units='$10eV$', isCommunicated=True)
rk.addVarAndDual("TePerp", TePerp, isStationary=True, units='$10eV$', isCommunicated=True)
rk.addVarAndDual("TiPar", TiPar, isStationary=True, units='$10eV$', isCommunicated=True)
rk.addVarAndDual("TiPerp", TiPerp, isStationary=True, units='$10eV$', isCommunicated=True)

## Anisotropic variables

In [7]:
rk.addVarAndDual("collFreq", np.ones(2),isDerived=True, isCommunicated=True)

In [8]:
rk.addVarAndDual("betaPare", isDerived=True, derivationRule=sc.derivationRule("betaPare",("TePar","TePar")),
                 derivOptions=treeDerivation(aniso.betasr("TePar","TePar",elMass,elMass)))
rk.addVarAndDual("betaPerpe", isDerived=True, derivationRule=sc.derivationRule("betaPerpe",("TePerp","TePerp")),
                 derivOptions=treeDerivation(aniso.betasr("TePerp","TePerp",elMass,elMass)))
rk.addVarAndDual("betaPari", isDerived=True, derivationRule=sc.derivationRule("betaPari",("TiPar","TiPar")),
                 derivOptions=treeDerivation(aniso.betasr("TiPar","TiPar",ionMass,ionMass)))
rk.addVarAndDual("betaPerpi", isDerived=True, derivationRule=sc.derivationRule("betaPerpi",("TiPerp","TiPerp")),
                 derivOptions=treeDerivation(aniso.betasr("TiPerp","TiPerp",ionMass,ionMass)))

In [9]:
rk.addVarAndDual("alphae", isDerived=True, derivationRule=sc.derivationRule("alphae",("TePar","TePerp","TePar","TePerp")),
                 derivOptions=treeDerivation(aniso.alphasr("TePar","TePerp","TePar","TePerp",elMass,elMass)))
rk.addVarAndDual("alphai", isDerived=True, derivationRule=sc.derivationRule("alphai",("TiPar","TiPerp","TiPar","TiPerp")),
                 derivOptions=treeDerivation(aniso.alphasr("TiPar","TiPerp","TiPar","TiPerp",ionMass,ionMass)))

rk.addCustomDerivation("K200e", treeDerivation(aniso.K_LMN(Node("alphae"),"200")))
rk.addCustomDerivation("K002e", treeDerivation(aniso.K_LMN(Node("alphae"),"002")))
rk.addCustomDerivation("K200i", treeDerivation(aniso.K_LMN(Node("alphai"),"200")))
rk.addCustomDerivation("K002i", treeDerivation(aniso.K_LMN(Node("alphai"),"002")))

rk.addCustomDerivation("K200eSmall", treeDerivation(aniso.K_LMN(Node("alphae"),"200",True)))
rk.addCustomDerivation("K002eSmall", treeDerivation(aniso.K_LMN(Node("alphae"),"002",True)))
rk.addCustomDerivation("K200iSmall", treeDerivation(aniso.K_LMN(Node("alphai"),"200",True)))
rk.addCustomDerivation("K002iSmall", treeDerivation(aniso.K_LMN(Node("alphai"),"002",True)))

In [10]:

rk.addCustomDerivation("filterSmallVals200e", sc.rangeFilterDerivation("K200eSmall",[1],[[-0.01,0.01]],[2]))
rk.addCustomDerivation("filterPlusLarge200e", sc.rangeFilterDerivation("K200e",[1],[[0.01,1e16]],[2]))
rk.addCustomDerivation("filterMinusLarge200e", sc.rangeFilterDerivation("K200e",[1],[[-1e16,-0.01]],[2]))

rk.addCustomDerivation("filterSmallVals002e", sc.rangeFilterDerivation("K002eSmall",[1],[[-0.01,0.01]],[2]))
rk.addCustomDerivation("filterPlusLarge002e", sc.rangeFilterDerivation("K002e",[1],[[0.01,1e16]],[2]))
rk.addCustomDerivation("filterMinusLarge002e", sc.rangeFilterDerivation("K002e",[1],[[-1e16,-0.01]],[2]))

rk.addCustomDerivation("filterSmallVals200i", sc.rangeFilterDerivation("K200iSmall",[1],[[-0.01,0.01]],[2]))
rk.addCustomDerivation("filterPlusLarge200i", sc.rangeFilterDerivation("K200i",[1],[[0.01,1e16]],[2]))
rk.addCustomDerivation("filterMinusLarge200i", sc.rangeFilterDerivation("K200i",[1],[[-1e16,-0.01]],[2]))

rk.addCustomDerivation("filterSmallVals002i", sc.rangeFilterDerivation("K002iSmall",[1],[[-0.01,0.01]],[2]))
rk.addCustomDerivation("filterPlusLarge002i", sc.rangeFilterDerivation("K002i",[1],[[0.01,1e16]],[2]))
rk.addCustomDerivation("filterMinusLarge002i", sc.rangeFilterDerivation("K002i",[1],[[-1e16,-0.01]],[2]))

In [11]:

rk.addVarAndDual("Xe", isDerived=True, derivationRule=sc.derivationRule("XeDeriv", ("TePar","TePerp","TePar","TePerp")),
                 derivOptions=treeDerivation(aniso.alphasr("TePar","TePerp","TePar","TePerp",elMass,elMass) - 1))

rk.addVarAndDual("Xi", isDerived=True, derivationRule=sc.derivationRule("XiDeriv", ("TiPar","TiPerp","TiPar","TiPerp")),
                 derivOptions=treeDerivation(aniso.alphasr("TiPar","TiPerp","TiPar","TiPerp",ionMass,ionMass) - 1))

rk.addVarAndDual("K200e", isDerived=True, derivationRule=sc.derivationRule("filteredK200e",["Xe","alphae"]),
                 derivOptions=sc.additiveDerivation(["filterSmallVals200e","filterPlusLarge200e","filterMinusLarge200e"],1.,[[1,2]]*3,[1,1,1]))

rk.addVarAndDual("K002e", isDerived=True, derivationRule=sc.derivationRule("filteredK002e",["Xe","alphae"]),
                 derivOptions=sc.additiveDerivation(["filterSmallVals002e","filterPlusLarge002e","filterMinusLarge002e"],1.,[[1,2]]*3,[1,1,1]))

rk.addVarAndDual("K200i", isDerived=True, derivationRule=sc.derivationRule("filteredK200i",["Xi","alphai"]),
                 derivOptions=sc.additiveDerivation(["filterSmallVals200i","filterPlusLarge200i","filterMinusLarge200i"],1.,[[1,2]]*3,[1,1,1]))

rk.addVarAndDual("K002i", isDerived=True, derivationRule=sc.derivationRule("filteredK002i",["Xi","alphai"]),
                 derivOptions=sc.additiveDerivation(["filterSmallVals002i","filterPlusLarge002i","filterMinusLarge002i"],1.,[[1,2]]*3,[1,1,1]))


## Implicit temperature derivation

In [12]:
# Implicit temperature equations

# Adding the model tag to tag list
modelTag = "implicitTempIonPar"

# Initializing model
implicitTempModelPar = cm.implicitTemperatures(modelTag=modelTag,
                                            speciesFluxes=["Gi_dual"],
                                            speciesDensities=["ni"],
                                            speciesEnergies=["WiPar"],
                                            speciesTemperatures=["TiPar"],
                                            species=[ionSpecies],
                                            speciesDensitiesDual=["ni_dual"],
                                            degreesOfFreedom=1)

rk.addModel(implicitTempModelPar)

# Adding the model tag to tag list
modelTag = "implicitTempIonPerp"

# Initializing model
implicitTempModelPerp = cm.implicitTemperatures(modelTag=modelTag,
                                            speciesFluxes=["Gi_dual"],
                                            speciesDensities=["ni"],
                                            speciesEnergies=["WiPerp"],
                                            speciesTemperatures=["TiPerp"],
                                            species=[ionSpecies],
                                            speciesDensitiesDual=["ni_dual"],
                                            degreesOfFreedom=2)

rk.addModel(implicitTempModelPerp)


Checking terms in model implicitTempIonPar:
   Checking term identityTermTiPar
   Checking term wTermTiPar
   Checking term u2TermTiPar
Checking terms in model implicitTempIonPerp:
   Checking term identityTermTiPerp
   Checking term wTermTiPerp
   Checking term u2TermTiPerp


In [13]:
# Implicit temperature equations

# Adding the model tag to tag list
modelTag = "implicitTempElPar"

# Initializing model
implicitTempModelPar = cm.implicitTemperatures(modelTag=modelTag,
                                            speciesFluxes=["Ge_dual"],
                                            speciesDensities=["ne"],
                                            speciesEnergies=["WePar"],
                                            speciesTemperatures=["TePar"],
                                            species=[electronSpecies],
                                            speciesDensitiesDual=["ne_dual"],
                                            degreesOfFreedom=1)

rk.addModel(implicitTempModelPar)

# Adding the model tag to tag list
modelTag = "implicitTempElPerp"

# Initializing model
implicitTempModelPerp = cm.implicitTemperatures(modelTag=modelTag,
                                            speciesFluxes=["Ge_dual"],
                                            speciesDensities=["ne"],
                                            speciesEnergies=["WePerp"],
                                            speciesTemperatures=["TePerp"],
                                            species=[electronSpecies],
                                            speciesDensitiesDual=["ne_dual"],
                                            degreesOfFreedom=2,
                                            ignoreKineticContribution=True)

rk.addModel(implicitTempModelPerp)

Checking terms in model implicitTempElPar:
   Checking term identityTermTePar
   Checking term wTermTePar
   Checking term u2TermTePar
Checking terms in model implicitTempElPerp:
   Checking term identityTermTePerp
   Checking term wTermTePerp


## Electron-ion energy exchange

In [14]:
perpElExch = cm.addNodeMatrixTermModel(rk, modelTag="perpElExch", evolvedVar="WePerp",
                                       termDefs=[(8*(elMass/ionMass)*Node("collFreq")*Node("ne")*Node("K200e"),"TiPerp"),
                                                 (-8*(elMass/ionMass)*Node("collFreq")*Node("ne")*Node("K200e"),"TePerp")])

parElExch = cm.addNodeMatrixTermModel(rk, modelTag="parElExch", evolvedVar="WePar",
                                      termDefs=[(8*(elMass/ionMass)*Node("collFreq")*Node("ne")*Node("alphae")*Node("K002e"),"TiPar"),
                                                (-8*(elMass/ionMass)*Node("collFreq")*Node("ne")*Node("alphae")*Node("K002e"),"TePar")])

perpIonExch = cm.addNodeMatrixTermModel(rk, modelTag="perpIonExch", evolvedVar="WiPerp",
                                        termDefs=[(8*(elMass/ionMass)*Node("collFreq")*Node("ni")*Node("K200e"),"TePerp"),
                                                  (-8*(elMass/ionMass)*Node("collFreq")*Node("ni")*Node("K200e"),"TiPerp")])

parIonExch = cm.addNodeMatrixTermModel(rk, modelTag="parIonExch", evolvedVar="WiPar",
                                      termDefs=[(8*(elMass/ionMass)*Node("collFreq")*Node("ni")*Node("alphae")*Node("K002e"),"TePar"),
                                                (-8*(elMass/ionMass)*Node("collFreq")*Node("ni")*Node("alphae")*Node("K002e"),"TiPar")])

Checking terms in model perpElExch:
   Checking term nodeTerm_0
   Checking term nodeTerm_1
Checking terms in model parElExch:
   Checking term nodeTerm_0
   Checking term nodeTerm_1
Checking terms in model perpIonExch:
   Checking term nodeTerm_0
   Checking term nodeTerm_1
Checking terms in model parIonExch:
   Checking term nodeTerm_0
   Checking term nodeTerm_1


## Temperature Isotropisation

In [15]:
perpIsoEl = cm.addNodeMatrixTermModel(rk,modelTag="perpIsoEl",evolvedVar="WePerp",
                                      termDefs=[(2*Node("collFreq")*Node("K002e")/Node("betaPerpe"),"ne"),
                                                (-2*Node("collFreq")*Node("K200e")/Node("betaPerpe"),"ne")])

parIsoEl = cm.addNodeMatrixTermModel(rk,modelTag="parIsoEl",evolvedVar="WePar",
                                     termDefs=[(4*Node("collFreq")*Node("alphae")*Node("K200e")/Node("betaPare"),"ne"),
                                               (-4*Node("collFreq")*Node("alphae")*Node("K002e")/Node("betaPare"),"ne")])

perpIsoIon = cm.addNodeMatrixTermModel(rk, modelTag="perpIsoIon",evolvedVar="WiPerp",
                                       termDefs=[(2*Node("collFreq")*Node("K002i")/Node("betaPerpi"),"ni"),
                                                (-2*Node("collFreq")*Node("K200i")/Node("betaPerpi"),"ni")])

parIsoIon = cm.addNodeMatrixTermModel(rk,modelTag="parIsoIon",evolvedVar="WiPar",
                                     termDefs=[(4*Node("collFreq")*Node("alphai")*Node("K200i")/Node("betaPari"),"ni"),
                                               (-4*Node("collFreq")*Node("alphai")*Node("K002i")/Node("betaPari"),"ni")])

Checking terms in model perpIsoEl:
   Checking term nodeTerm_0
   Checking term nodeTerm_1
Checking terms in model parIsoEl:
   Checking term nodeTerm_0
   Checking term nodeTerm_1
Checking terms in model perpIsoIon:
   Checking term nodeTerm_0
   Checking term nodeTerm_1
Checking terms in model parIsoIon:
   Checking term nodeTerm_0
   Checking term nodeTerm_1


### Integrator options

In [16]:
integrator = sc.picardBDEIntegrator(absTol=100.0, convergenceVars=["WePar","WePerp", "WiPar", "WiPerp"],internalStepControl=True)

rk.setPETScOptions(cliOpts="-pc_type bjacobi -sub_pc_factor_shift_type nonzero -sub_pc_factor_levels 1 -ksp_gmres_restart 200",kspSolverType="gmres")

rk.addIntegrator("BE", integrator)

initialTimestep = 10
rk.setIntegratorGlobalData(3, 2, initialTimestep)

rk.addTermDiagnosisForVars(["WePar","WePerp","WiPar","WiPerp"])

bdeStep = sc.IntegrationStep("BE", defaultEvaluateGroups=[
                             1, 2, 3], defaultUpdateModelData=True, defaultUpdateGroups=[1, 2, 3])

rk.addVar("scaling", isDerived=True, derivationRule=sc.derivationRule("scaling",["Xe"]), derivOptions=treeDerivation(1.0/(1.0+1000*erf(Node("Xe")))))
rk.setTimestepController(sc.scalingTimestepController(reqVarNames=["scaling"],reqVarPowers=[1.0]))

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE", bdeStep.dict())

rk.setFixedNumTimesteps(200)
rk.setFixedStepOutput(2)
rk.setRestartOptions(False, False, 1000) #Change to True when restarting

## Write config file

In [17]:
rk.writeConfigFile()

## Data analysis

In [ ]:
numFiles = 100
loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames, isXinMeters=rk.grid)
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline
plt.rcParams['figure.dpi'] = 150
hv.output(size=100, dpi=150)

In [ ]:
plt.figure(figsize=(4,4))
plt.semilogx(loadedData.coords["time"],loadedData["TePar"][:,1],label="TePar")
plt.semilogx(loadedData.coords["time"],loadedData["TePerp"][:,1],label="TePerp")
plt.semilogx(loadedData.coords["time"],loadedData["TiPar"][:,1],label="TiPar")
plt.semilogx(loadedData.coords["time"],loadedData["TiPerp"][:,1],label="TiPerp")
plt.title("Energy Exchange and Temperature Isotropisation\nof Electrons and Ions")
plt.xlabel("Time (arbitrary units)")
plt.ylabel("Temperature (arbitrary units)")
plt.legend()
plt.show()